# Country Clustering with PCA, K-Means, DBSCAN and HDBSCAN  
### Unsupervised Learning – Economic Segmentation of Countries

In this notebook, we explore unsupervised learning algorithms to segment countries based on socio-economic indicators.  
We use multiple clustering methods:

- **K-Means**
- **Hierarchical Clustering**
- **DBSCAN**
- **HDBSCAN**

We also apply **PCA** for dimensionality reduction and visualize results on a **world map**.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import ceil
import seaborn as sns
import plotly_express as px

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans,DBSCAN
import hdbscan
from scipy.cluster.hierarchy import linkage,fcluster
from sklearn.metrics import silhouette_score

## 1. Load Dataset

We begin by loading the unsupervised learning dataset containing economic indicators for various countries.

In [2]:
df = pd.read_csv("/kaggle/input/unsupervised-learning-on-country-data/Country-data.csv")

print(df.head())

               country  child_mort  exports  health  imports  income  \
0          Afghanistan        90.2     10.0    7.58     44.9    1610   
1              Albania        16.6     28.0    6.55     48.6    9930   
2              Algeria        27.3     38.4    4.17     31.4   12900   
3               Angola       119.0     62.3    2.85     42.9    5900   
4  Antigua and Barbuda        10.3     45.5    6.03     58.9   19100   

   inflation  life_expec  total_fer   gdpp  
0       9.44        56.2       5.82    553  
1       4.49        76.3       1.65   4090  
2      16.10        76.5       2.89   4460  
3      22.40        60.1       6.16   3530  
4       1.44        76.8       2.13  12200  


In [3]:
print(df.columns)

Index(['country', 'child_mort', 'exports', 'health', 'imports', 'income',
       'inflation', 'life_expec', 'total_fer', 'gdpp'],
      dtype='object')


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     167 non-null    object 
 1   child_mort  167 non-null    float64
 2   exports     167 non-null    float64
 3   health      167 non-null    float64
 4   imports     167 non-null    float64
 5   income      167 non-null    int64  
 6   inflation   167 non-null    float64
 7   life_expec  167 non-null    float64
 8   total_fer   167 non-null    float64
 9   gdpp        167 non-null    int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 13.2+ KB
None


## 2. Preprocessing

We drop the country names and scale all numerical features.  
Scaling is crucial before PCA and distance-based clustering algorithms.

In [5]:
df2 = df.drop(columns=["country"],axis=1)
scaler = StandardScaler()
df2 = scaler.fit_transform(df2)

df2 = pd.DataFrame(df2, columns=[
    'child_mort','exports','health','imports','income',
    'inflation','life_expec','total_fer','gdpp'
])

## 3. PCA Function

We create a reusable PCA function that returns transformed components along with explained variance.

In [6]:
def apply_pca(df2, n_components=3, random_state=13):
    pca = PCA(n_components=n_components, random_state=random_state)
    transformed_pca = pca.fit_transform(df2)

    pca_df = pd.DataFrame(transformed_pca, columns=[f"PC{i+1}" for i in range(n_components)])

    explained_var = np.sum(pca.explained_variance_ratio_) * 100
    print(f"PCA applied. {n_components} components explain {explained_var:.2f}% variance.")

    return pca_df

## 4. Unified Clustering Function

We wrap all clustering algorithms into a single function:

- `kmeans`
- `hierarchical`
- `dbscan`
- `hdbscan`

This function:
1. Applies PCA (optional)
2. Performs clustering
3. Computes silhouette score
4. Assigns cluster labels
5. Creates a world map visualization

In [7]:
def run_clustering(algorithm, df_original, use_pca=True, n_pca=3):
    df_copy = df_original.copy()
    data = df2.copy()

    if use_pca:
        print("Applying PCA...")
        data = apply_pca(data, n_components=n_pca)

    if algorithm == "kmeans":
        model = KMeans(n_clusters=3, random_state=15)
        labels = model.fit_predict(data)

    elif algorithm == "hierarchical":
        Z = linkage(data, method="ward")
        labels = fcluster(Z, t=3, criterion="maxclust")

    elif algorithm == "dbscan":
        model = DBSCAN(eps=1.0, min_samples=3)
        labels = model.fit_predict(data)

    elif algorithm == "hdbscan":
        model = hdbscan.HDBSCAN(min_cluster_size=3)
        labels = model.fit_predict(data)

    else:
        raise ValueError("Algorithm name must be one of: kmeans, hierarchical, dbscan, hdbscan")

    if len(set(labels)) > 1 and len(df_copy) > len(set(labels)):
        score = silhouette_score(data, labels)
    else:
        score = None

    print(f"{algorithm.upper()} → Clusters: {np.unique(labels)} | Silhouette Score: {score}")

    df_copy["Class"] = labels

    valid = df_copy[df_copy["Class"] != -1]
    if len(valid["Class"].unique()) > 1:
        cluster_summary = valid.groupby("Class")["income"].mean().sort_values()

        mapping = {}
        labels_list = ["Budget Needed", "In Between", "No Budget Needed"]
        for i, idx in enumerate(cluster_summary.index):
            label_name = labels_list[i % len(labels_list)]
            mapping[idx] = label_name

        df_copy["Class_Label"] = df_copy["Class"].map(mapping)

    df_copy["Class_Label"] = df_copy["Class_Label"].fillna("Noise")

    title_map = {
        "kmeans": "KMeans Clustering",
        "hierarchical": "Hierarchical Clustering",
        "dbscan": "DBSCAN Density-Based Clustering",
        "hdbscan": "HDBSCAN Hierarchical Density Clustering",
    }

    title_suffix = "with PCA" if use_pca else "without PCA"

    fig = px.choropleth(
        df_copy,
        locations="country",
        locationmode="country names",
        color="Class_Label",
        title=f"{title_map[algorithm]} ({title_suffix}) — Economic Segmentation",
        color_discrete_map={
            "Budget Needed": "red",
            "In Between": "yellow",
            "No Budget Needed": "green",
            "Noise": "gray"
        }
    )
    fig.update_geos(fitbounds="locations", visible=True)
    fig.show()

    return df_copy, score

## 🔹 KMeans Clustering (with PCA)

We first apply PCA to reduce dimensionality to 3 components,  
then run **KMeans** clustering.  
This helps improve cluster separation and reduces noise.

In [8]:
run_clustering(
    algorithm="kmeans",
    df_original=df,
    use_pca=True,
    n_pca=3
)

Applying PCA...
PCA applied. 3 components explain 76.14% variance.
KMEANS → Clusters: [0 1 2] | Silhouette Score: 0.3721440572631934


/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


(                 country  child_mort  exports  health  imports  income  \
 0            Afghanistan        90.2     10.0    7.58     44.9    1610   
 1                Albania        16.6     28.0    6.55     48.6    9930   
 2                Algeria        27.3     38.4    4.17     31.4   12900   
 3                 Angola       119.0     62.3    2.85     42.9    5900   
 4    Antigua and Barbuda        10.3     45.5    6.03     58.9   19100   
 ..                   ...         ...      ...     ...      ...     ...   
 162              Vanuatu        29.2     46.6    5.25     52.7    2950   
 163            Venezuela        17.1     28.5    4.91     17.6   16500   
 164              Vietnam        23.3     72.0    6.84     80.2    4490   
 165                Yemen        56.3     30.0    5.18     34.4    4480   
 166               Zambia        83.1     37.0    5.89     30.9    3280   
 
      inflation  life_expec  total_fer   gdpp  Class    Class_Label  
 0         9.44        56.2 

## 🔹 KMeans Clustering (without PCA)

Here we directly cluster using raw scaled features.  
This allows us to compare how PCA affects performance.

In [9]:
run_clustering(
    algorithm="kmeans",
    df_original=df,
    use_pca=False
)

KMEANS → Clusters: [0 1 2] | Silhouette Score: 0.28329575683463126


/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



(                 country  child_mort  exports  health  imports  income  \
 0            Afghanistan        90.2     10.0    7.58     44.9    1610   
 1                Albania        16.6     28.0    6.55     48.6    9930   
 2                Algeria        27.3     38.4    4.17     31.4   12900   
 3                 Angola       119.0     62.3    2.85     42.9    5900   
 4    Antigua and Barbuda        10.3     45.5    6.03     58.9   19100   
 ..                   ...         ...      ...     ...      ...     ...   
 162              Vanuatu        29.2     46.6    5.25     52.7    2950   
 163            Venezuela        17.1     28.5    4.91     17.6   16500   
 164              Vietnam        23.3     72.0    6.84     80.2    4490   
 165                Yemen        56.3     30.0    5.18     34.4    4480   
 166               Zambia        83.1     37.0    5.89     30.9    3280   
 
      inflation  life_expec  total_fer   gdpp  Class    Class_Label  
 0         9.44        56.2 

## 🔹 Hierarchical Clustering (with PCA)

We apply PCA first to reduce noise and correlation,  
then perform **Ward linkage hierarchical clustering**.

In [10]:
run_clustering(
    algorithm="hierarchical",
    df_original=df,
    use_pca=True,
    n_pca=3
)

Applying PCA...
PCA applied. 3 components explain 76.14% variance.
HIERARCHICAL → Clusters: [1 2 3] | Silhouette Score: 0.3260580737036453


(                 country  child_mort  exports  health  imports  income  \
 0            Afghanistan        90.2     10.0    7.58     44.9    1610   
 1                Albania        16.6     28.0    6.55     48.6    9930   
 2                Algeria        27.3     38.4    4.17     31.4   12900   
 3                 Angola       119.0     62.3    2.85     42.9    5900   
 4    Antigua and Barbuda        10.3     45.5    6.03     58.9   19100   
 ..                   ...         ...      ...     ...      ...     ...   
 162              Vanuatu        29.2     46.6    5.25     52.7    2950   
 163            Venezuela        17.1     28.5    4.91     17.6   16500   
 164              Vietnam        23.3     72.0    6.84     80.2    4490   
 165                Yemen        56.3     30.0    5.18     34.4    4480   
 166               Zambia        83.1     37.0    5.89     30.9    3280   
 
      inflation  life_expec  total_fer   gdpp  Class    Class_Label  
 0         9.44        56.2 

## 🔹 Hierarchical Clustering (without PCA)

Hierarchical clustering on the full feature set.  
Useful for seeing natural tree-based structure without dimensionality reduction.

In [11]:
run_clustering(
    algorithm="hierarchical",
    df_original=df,
    use_pca=False
)

HIERARCHICAL → Clusters: [1 2 3] | Silhouette Score: 0.24563001303300652


(                 country  child_mort  exports  health  imports  income  \
 0            Afghanistan        90.2     10.0    7.58     44.9    1610   
 1                Albania        16.6     28.0    6.55     48.6    9930   
 2                Algeria        27.3     38.4    4.17     31.4   12900   
 3                 Angola       119.0     62.3    2.85     42.9    5900   
 4    Antigua and Barbuda        10.3     45.5    6.03     58.9   19100   
 ..                   ...         ...      ...     ...      ...     ...   
 162              Vanuatu        29.2     46.6    5.25     52.7    2950   
 163            Venezuela        17.1     28.5    4.91     17.6   16500   
 164              Vietnam        23.3     72.0    6.84     80.2    4490   
 165                Yemen        56.3     30.0    5.18     34.4    4480   
 166               Zambia        83.1     37.0    5.89     30.9    3280   
 
      inflation  life_expec  total_fer   gdpp  Class    Class_Label  
 0         9.44        56.2 

## 🔹 DBSCAN (with PCA)

DBSCAN is sensitive to dimensionality.  
Using PCA greatly improves cluster detection and noise handling.

In [12]:
run_clustering(
    algorithm="dbscan",
    df_original=df,
    use_pca=True,
    n_pca=3
)

Applying PCA...
PCA applied. 3 components explain 76.14% variance.
DBSCAN → Clusters: [-1  0  1  2] | Silhouette Score: -0.014042075846429044


(                 country  child_mort  exports  health  imports  income  \
 0            Afghanistan        90.2     10.0    7.58     44.9    1610   
 1                Albania        16.6     28.0    6.55     48.6    9930   
 2                Algeria        27.3     38.4    4.17     31.4   12900   
 3                 Angola       119.0     62.3    2.85     42.9    5900   
 4    Antigua and Barbuda        10.3     45.5    6.03     58.9   19100   
 ..                   ...         ...      ...     ...      ...     ...   
 162              Vanuatu        29.2     46.6    5.25     52.7    2950   
 163            Venezuela        17.1     28.5    4.91     17.6   16500   
 164              Vietnam        23.3     72.0    6.84     80.2    4490   
 165                Yemen        56.3     30.0    5.18     34.4    4480   
 166               Zambia        83.1     37.0    5.89     30.9    3280   
 
      inflation  life_expec  total_fer   gdpp  Class Class_Label  
 0         9.44        56.2    

## 🔹 DBSCAN (without PCA)

Raw feature space is used.  
Useful for testing density-based clustering performance without preprocessing.

In [13]:
run_clustering(
    algorithm="dbscan",
    df_original=df,
    use_pca=False
)

DBSCAN → Clusters: [-1  0  1  2  3  4  5] | Silhouette Score: 0.011714126637591596


(                 country  child_mort  exports  health  imports  income  \
 0            Afghanistan        90.2     10.0    7.58     44.9    1610   
 1                Albania        16.6     28.0    6.55     48.6    9930   
 2                Algeria        27.3     38.4    4.17     31.4   12900   
 3                 Angola       119.0     62.3    2.85     42.9    5900   
 4    Antigua and Barbuda        10.3     45.5    6.03     58.9   19100   
 ..                   ...         ...      ...     ...      ...     ...   
 162              Vanuatu        29.2     46.6    5.25     52.7    2950   
 163            Venezuela        17.1     28.5    4.91     17.6   16500   
 164              Vietnam        23.3     72.0    6.84     80.2    4490   
 165                Yemen        56.3     30.0    5.18     34.4    4480   
 166               Zambia        83.1     37.0    5.89     30.9    3280   
 
      inflation  life_expec  total_fer   gdpp  Class Class_Label  
 0         9.44        56.2    

## 🔹 HDBSCAN (with PCA)

HDBSCAN excels when PCA is applied because dimensionality reduction  
stabilizes density regions and improves cluster discoverability.

In [14]:
run_clustering(
    algorithm="hdbscan",
    df_original=df,
    use_pca=True,
    n_pca=3
)

Applying PCA...
PCA applied. 3 components explain 76.14% variance.
HDBSCAN → Clusters: [-1  0  1] | Silhouette Score: -0.006817701111461716


(                 country  child_mort  exports  health  imports  income  \
 0            Afghanistan        90.2     10.0    7.58     44.9    1610   
 1                Albania        16.6     28.0    6.55     48.6    9930   
 2                Algeria        27.3     38.4    4.17     31.4   12900   
 3                 Angola       119.0     62.3    2.85     42.9    5900   
 4    Antigua and Barbuda        10.3     45.5    6.03     58.9   19100   
 ..                   ...         ...      ...     ...      ...     ...   
 162              Vanuatu        29.2     46.6    5.25     52.7    2950   
 163            Venezuela        17.1     28.5    4.91     17.6   16500   
 164              Vietnam        23.3     72.0    6.84     80.2    4490   
 165                Yemen        56.3     30.0    5.18     34.4    4480   
 166               Zambia        83.1     37.0    5.89     30.9    3280   
 
      inflation  life_expec  total_fer   gdpp  Class    Class_Label  
 0         9.44        56.2 

## 🔹 HDBSCAN (without PCA)

Clustering is performed on the complete scaled dataset.  
Useful for comparing how PCA influences cluster stability and noise labeling.

In [15]:
run_clustering(
    algorithm="hdbscan",
    df_original=df,
    use_pca=False
)

HDBSCAN → Clusters: [-1  0  1  2] | Silhouette Score: 0.10160882853966124


(                 country  child_mort  exports  health  imports  income  \
 0            Afghanistan        90.2     10.0    7.58     44.9    1610   
 1                Albania        16.6     28.0    6.55     48.6    9930   
 2                Algeria        27.3     38.4    4.17     31.4   12900   
 3                 Angola       119.0     62.3    2.85     42.9    5900   
 4    Antigua and Barbuda        10.3     45.5    6.03     58.9   19100   
 ..                   ...         ...      ...     ...      ...     ...   
 162              Vanuatu        29.2     46.6    5.25     52.7    2950   
 163            Venezuela        17.1     28.5    4.91     17.6   16500   
 164              Vietnam        23.3     72.0    6.84     80.2    4490   
 165                Yemen        56.3     30.0    5.18     34.4    4480   
 166               Zambia        83.1     37.0    5.89     30.9    3280   
 
      inflation  life_expec  total_fer   gdpp  Class    Class_Label  
 0         9.44        56.2 

# 🏁 Conclusion

In this notebook, we explored multiple unsupervised learning algorithms to segment countries based on socio-economic indicators such as income, child mortality, inflation, GDP, and health expenditure.  

Our key findings:

### 🔹 PCA significantly improves clustering stability  
Dimensionality reduction with **3 principal components** provided clearer cluster boundaries and reduced noise for most algorithms, especially **KMeans** and **Hierarchical Clustering**.

### 🔹 KMeans delivered the best overall performance  
- Highest Silhouette Score with PCA  
- Stable number of clusters  
- Clear separation between income groups  
This makes KMeans a strong baseline for economic segmentation tasks.

### 🔹 Density-based methods struggled with high-dimensional data  
- **DBSCAN** and **HDBSCAN** produced many noise samples  
- PCA helped, but improvements were still limited  
These algorithms may require more careful parameter tuning or alternative distance metrics.

### 🔹 PCA-supported clustering captured meaningful economic patterns  
Low-income countries tended to cluster together into the “Budget Needed” class,  
while high-income countries formed the “No Budget Needed” segment.  
This suggests that socio-economic indicators naturally form structured patterns suitable for unsupervised learning.

Overall, combining **PCA + KMeans** proved to be the most effective strategy for this dataset.


# 🔮 Future Work

There are several promising directions to further improve this analysis:

### ➤ 1. Hyperparameter Optimization  
Automated tuning methods (GridSearch, Bayesian Optimization) could improve:
- DBSCAN `eps` and `min_samples`
- HDBSCAN `min_cluster_size`
- KMeans `n_clusters`

### ➤ 2. Alternative Dimensionality Reduction  
Other techniques may reveal richer structure:
- **t-SNE**
- **UMAP**
- **Kernel PCA**

These methods often capture non-linear relationships better than PCA.

### ➤ 3. Feature Engineering  
Creating new indicators such as:
- GDP per capita growth  
- Income-to-health ratio  
- Composite development indices  
may enhance cluster interpretability.

### ➤ 4. Cluster Profiling with Additional Datasets  
Integrating external sources (World Bank, UN Data) can help analyze:
- Human Development Index (HDI)
- Education levels
- Inequality metrics

This would provide deeper context to each cluster.

### ➤ 5. Model Deployment for Policy Insights  
A dashboard or API could be created to:
- Assign new countries to clusters  
- Visualize global development categories  
- Support economic planning and funding strategies  


This notebook provides a strong foundation for unsupervised country segmentation and can be extended into a full analytical pipeline.
